## Procedure for generating the random cells
1. Optimise the geometries of the b-Sn, sh, and cd structures of Si (get from mp)
2. make supercells (roughly 64 atoms)
3. rattle by 0.01 eV and 3% (positions and lattice parameters respectively)

In [119]:
import pymatgen.ext.matproj as mp
import pymatgen.core.periodic_table as pt
import pymatgen.io.ase as pase
from ase.atoms import Atoms
from ase.io.castep import write_castep_cell, read_castep_geom
import os
from ase import build
import numpy as np
from numpy.random import normal
from copy import deepcopy

In [102]:
os.chdir('/home/joe/Documents/Ge_od/Structure_databases/crystals/Si_crystals_opt')

Getting structures from the materials project

In [120]:
with mp.MPRester("v0JVgKLlpTXKEhiKeDIC") as m:
    results = m.query(criteria={"pretty_formula":'Si'},
                            properties=["exp.tags", "icsd_ids", "spacegroup",
                                        "task_id", "density", "final_structure",
                                        "conventional_unit_cell"])
    #results = [m.get_structure_by_material_id(i, conventional_unit_cell=True) for i in ['mp-78', 'mp-1061054']]

In [30]:
for i, val in enumerate(results):
    print(val['task_id'], val['spacegroup']['symbol'], i)

mp-1196961 P6/mmm 0
mp-149 Fd-3m 1
mp-1072544 Im-3m 2
mp-1204046 P4_2/nmc 3
mp-1244971 P1 4
mp-971661 I-43m 5
mp-1120747 P6_3/mmc 6
mp-1095269 Cmcm 7
mp-1199894 C2/c 8
mp-1079649 Cmce 9
mp-10649 P6_3/mmc 10
mp-1094057 R-3m 11
mp-1268191 P6/m 12
mp-1204627 Cmcm 13
mp-1120447 P1 14
mp-27 Fm-3m 15
mp-571520 R-3 16
mp-1245041 P1 17
mp-999200 I4/mmm 18
mp-644693 P1 19
mp-1202745 P-3m1 20
mp-971662 Pm-3n 21
mp-1201492 Pa-3 22
mp-1200830 I-43m 23
mp-109 Fmmm 24
mp-1244933 P1 25
mp-92 I4_1/amd 26
mp-1244990 P1 27
mp-1001113 Cmcm 28
mp-34 P6/mmm 29
mp-676011 P1 30
mp-168 Ia-3 31
mp-1203790 P6_3/mmc 32
mp-1056579 I4/mmm 33
mp-165 P6_3/mmc 34
mp-1079297 C2/m 35
mp-1014212 Im-3m 36
mp-1245242 P1 37
mp-16220 Fd-3m 38


In [35]:
results[29]

{'exp.tags': ['Silicon - HP',
  'Silicon - V, HP',
  'High pressure experimental phase'],
 'icsd_ids': [109035, 52456],
 'spacegroup': {'symprec': 0.1,
  'source': 'spglib',
  'symbol': 'P6/mmm',
  'number': 191,
  'point_group': '6/mmm',
  'crystal_system': 'hexagonal',
  'hall': '-P 6 2'},
 'task_id': 'mp-34',
 'density': 3.097101142265708,
 'final_structure': Structure Summary
 Lattice
     abc : 2.6495247864060456 2.6495247864060456 2.476905
  angles : 90.0 90.0 120.00001963680538
  volume : 15.05829736024591
       A : 1.324762 -2.294556 0.0
       B : 1.324762 2.294556 0.0
       C : 0.0 0.0 2.476905
 PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000],
 'conventional_unit_cell': None}

In [23]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


In [121]:
dia = pase.AseAtomsAdaptor().get_atoms(
    SpacegroupAnalyzer(results[1]['final_structure']).get_conventional_standard_structure())

In [122]:
bSn = pase.AseAtomsAdaptor().get_atoms(
    SpacegroupAnalyzer(results[26]['final_structure']).get_conventional_standard_structure())

In [123]:
sh = pase.AseAtomsAdaptor().get_atoms(
    SpacegroupAnalyzer(results[29]['final_structure']).get_conventional_standard_structure())

In [44]:
with open('dia_mp.cell', 'w') as f:
    write_castep_cell(f, dia)
    f.write('kpoints_mp_spacing : 0.05\n')
with open('bSn_mp.cell', 'w') as f:
    write_castep_cell(f, bSn)
    f.write('kpoints_mp_spacing : 0.05\n')
with open('sh_mp.cell', 'w') as f:
    write_castep_cell(f, sh)
    f.write('kpoints_mp_spacing : 0.05\n')

Now optimizing with castep on arcus-b at

/data/chem-amais/hert5155/Ge/crystal_structures/Si_crystals_opt

In [107]:
opts = []
for i in (names := list(reversed([i for i in os.listdir() if 'geom' in i]))):
    with open(i, 'r') as f:
        opts.append(read_castep_geom(f, index=-1))

Rattling:

In [64]:
def get_supercell_factor(at, target_size=64):
    f = np.ceil((target_size/len(at))**(1/3))
    print('supercell factor: %f leads to %f atoms' % (f, len(at)*f**3))
    return f

In [92]:
ncopies = 10
supers = [[] for i in opts]
for i, val in enumerate(opts):
    for j in range(ncopies):
        s = build.supercells.make_supercell(
                val, get_supercell_factor(val)*np.identity(3))
        s.rattle(0.01)
        s.set_cell(s.get_cell_lengths_and_angles() * np.concatenate([normal(1, 0.03, (3)), normal(1, 0.01, (3))]),
                 scale_atoms=True)
        supers[i].append(s)

supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
sup

In [93]:
supers[0][2].get_cell_lengths_and_angles()

array([10.78279457, 11.01779925, 11.12897889, 91.30962427, 88.54880661,
       90.59960994])

In [108]:
os.chdir('supercells')

In [109]:
ct = 0
for i in range(len(opts)):
    for j in supers[i]:
        ct += 1
        with open(str(ct)+'.cell', 'w') as f:
            f.write('# distorted supercell\n'
                    '# std dev pos: 0.01\n'
                    '# std dev cell vectors: 0.03\n'
                    '# std dev cell angles 0.01\n'
                    '# structure %s\n' % names[i])
            write_castep_cell(f, j)
            f.write('kpoints_mp_spacing 0.03\n')

Repeating for Ge (using Si data, scaled by 1.042)

In [127]:
os.chdir('/home/joe/Documents/Ge_od/Structure_databases/crystals/Ge_crystals_opt')

In [129]:
Ge_dia =  deepcopy(dia)
Ge_dia.set_atomic_numbers([32] * len(dia))
Ge_dia.set_cell(Ge_dia.get_cell()*1.042, scale_atoms=True)

In [130]:
Ge_bSn =  deepcopy(bSn)
Ge_bSn.set_atomic_numbers([32] * len(bSn))
Ge_bSn.set_cell(Ge_bSn.get_cell()*1.042, scale_atoms=True)

In [131]:
Ge_sh =  deepcopy(sh)
Ge_sh.set_atomic_numbers([32] * len(sh))
Ge_sh.set_cell(Ge_sh.get_cell()*1.042, scale_atoms=True)

In [132]:
with open('dia_mp.cell', 'w') as f:
    write_castep_cell(f, Ge_dia)
    f.write('kpoints_mp_spacing : 0.05\n')
with open('bSn_mp.cell', 'w') as f:
    write_castep_cell(f, Ge_bSn)
    f.write('kpoints_mp_spacing : 0.05\n')
with open('sh_mp.cell', 'w') as f:
    write_castep_cell(f, Ge_sh)
    f.write('kpoints_mp_spacing : 0.05\n')

Now optimizing with castep on arcus-b at

/data/chem-amais/hert5155/Ge/crystal_structures/Ge_crystals_opt

In [133]:
opts = []
for i in (names := list(reversed([i for i in os.listdir() if 'geom' in i]))):
    with open(i, 'r') as f:
        opts.append(read_castep_geom(f, index=-1))

In [ ]:
opts

Rattling:

In [134]:
ncopies = 10
supers = [[] for i in opts]
for i, val in enumerate(opts):
    for j in range(ncopies):
        s = build.supercells.make_supercell(
                val, get_supercell_factor(val)*np.identity(3))
        s.rattle(0.01)
        s.set_cell(s.get_cell_lengths_and_angles() * np.concatenate([normal(1, 0.03, (3)), normal(1, 0.01, (3))]),
                 scale_atoms=True)
        supers[i].append(s)

supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 2.000000 leads to 64.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
supercell factor: 3.000000 leads to 108.000000 atoms
sup

In [135]:
supers[0][2].get_cell_lengths_and_angles()

array([11.2652023 , 11.22931158, 11.43911201, 89.13448742, 90.3006825 ,
       88.91255662])

In [136]:
os.chdir('supercells')

In [137]:
ct = 0
for i in range(len(opts)):
    for j in supers[i]:
        ct += 1
        with open(str(ct)+'.cell', 'w') as f:
            f.write('# distorted supercell\n'
                    '# std dev pos: 0.01\n'
                    '# std dev cell vectors: 0.03\n'
                    '# std dev cell angles 0.01\n'
                    '# structure %s\n' % names[i])
            write_castep_cell(f, j)
            f.write('kpoints_mp_spacing 0.03\n')